# DEM - hole filling and ocean masking
A notebook filtering a DEM with a Gaussian filter for hole filling, and using a shapefile for defining the ocean. Plots are used to demonstrates several challenges with filtering in plots.

In [ ]:
import rioxarray
import xarray
import geopandas
import shapely.geometry
import rasterio
import rasterio.mask
import numpy
import matplotlib.pyplot
import scipy.ndimage
import scipy.interpolate
import pathlib

# Setup
### File names

In [ ]:
base_path = pathlib.Path(r'C:\Users\pearsonra\Documents\data\herbert_10res')
dem_path = pathlib.Path(r'dem')
dem_name = pathlib.Path(r'dem_filled_and_fake_bathymetry.nc')
model_path = pathlib.Path(r'flood_model')
injection_1_name = pathlib.Path(r'River1Dis.txt')
injection_2_name = pathlib.Path(r'River2Dis.txt')
param_name = pathlib.Path(r'BG_param.txt')

### Load in data
This is the DEM prepared in the **"Example_DEM_Shapefile_Filtering"** notebook. Convert from a DataSet to a DataArray for easy viewing. 

In [ ]:
dem = rioxarray.rioxarray.open_rasterio(base_path/dem_path/dem_name) # , masked=True
dem = dem.drop_vars(["original_z"])
dem = dem.squeeze()
dem = dem.to_array()
dem = dem.squeeze()

### View DEM and select injection points
Zoom in on the head waters and select injection points.

In [ ]:
f = matplotlib.pyplot.figure(figsize=(15, 5))
gs = f.add_gridspec(1, 3)

ax1 = f.add_subplot(gs[0, 0:2]) 
dem.plot(cmap="viridis", ax=ax1) # Greys_r

ax2 = f.add_subplot(gs[0,2])
dem.plot(cmap="viridis", ax=ax2, vmin = 0, vmax = 100)
matplotlib.pyplot.xlim([1.4237e6, 1.4285e6])

ax1.set_title("Filled DEM with faked Bathymetry")
ax2.set_title("Filled and Faked DEM Zoomed on head waters")

### Zoom in on injection sites
Set injection sites slightly down-stream to avoid upstream spillover as the DEM is not ringed 
The DEM zoomed in on the region to manually create a polygon here

In [ ]:
f = matplotlib.pyplot.figure(figsize=(15, 5))

ax1 = f.add_subplot(121) 
dem.plot(cmap="viridis", ax=ax1, vmin = 0, vmax = 100)
ax1.set_title("LiDAR Herbert - zoomed in on coast")
matplotlib.pyplot.xlim([1.4238e6, 1.425e6])
matplotlib.pyplot.ylim([4.9880e6, 4.9895e6])

ax2 = f.add_subplot(122) 
dem.plot(cmap="viridis", ax=ax2, vmin = 0, vmax = 100)
ax2.set_title("LiDAR Herbert - zoomed in on coast")
matplotlib.pyplot.xlim([1.4244e6, 1.4255e6])
matplotlib.pyplot.ylim([4.9835e6, 4.9855e6])

matplotlib.pyplot.show()

### Manually create a polygons

In [ ]:
injection_1_x1 = 1.42425e6
injection_1_x2 = 1.42439e6
injection_1_y1 = 4.9886e6
injection_1_y2 = 4.98845e6
injection_1 = shapely.geometry.Polygon([(injection_1_x1, injection_1_y1), (injection_1_x2, injection_1_y1),
                                        (injection_1_x2, injection_1_y2), (injection_1_x1, injection_1_y2)])
injection_1_gdf = geopandas.GeoSeries([injection_1])


injection_2_x1 = 1.42507e6
injection_2_x2 = 1.42518e6
injection_2_y1 = 4.9841e6
injection_2_y2 = 4.98425e6
injection_2 = shapely.geometry.Polygon([(injection_2_x1, injection_2_y1), (injection_2_x2, injection_2_y1),
                                        (injection_2_x2, injection_2_y2), (injection_2_x1, injection_2_y2)])
injection_2_gdf = geopandas.GeoSeries([injection_2])

#### Plot both together

In [ ]:
f = matplotlib.pyplot.figure(figsize=(15, 5))

ax1 = f.add_subplot(121) 
dem.plot(cmap="viridis", ax=ax1, vmin = 0, vmax = 65)
ax1.set_title("LiDAR Herbert - zoomed in on coast")
matplotlib.pyplot.xlim([1.4238e6, 1.425e6])
matplotlib.pyplot.ylim([4.9880e6, 4.9895e6])
injection_1_gdf.plot(color='None', edgecolor='red', linewidth=2, ax=ax1, zorder=4)

ax2 = f.add_subplot(122) 
dem.plot(cmap="viridis", ax=ax2, vmin = 0, vmax = 65)
ax2.set_title("LiDAR Herbert - zoomed in on coast")
matplotlib.pyplot.xlim([1.4244e6, 1.4255e6])
matplotlib.pyplot.ylim([4.9835e6, 4.9855e6])
injection_2_gdf.plot(color='None', edgecolor='red', linewidth=2, ax=ax2, zorder=4)

matplotlib.pyplot.show()

# Save injection files
Example syntax below:

```
t0, flow_rate_0
t1, flow_rate_1
tn, flow_rate_2
```

Where the time must increase, but does not need to be evenly spaced.

In [ ]:
if not ((base_path/model_path).exists()):
    (base_path/model_path).mkdir()

In [ ]:
lines = ["# Herbet river. North branch. Injection rates\n", "0.0, 100.0\n", "600.0, 300.0\n", "10800.0, 300.0\n"] 
injection_file = open(base_path/model_path/injection_1_name,"w")
injection_file.writelines(lines)
injection_file.close()

In [ ]:
lines = ["# Herbet river. South branch. Injection rates\n", "0.0, 150.0\n", "600.0, 400.0\n", "10800.0, 400.0\n"] 
injection_file = open(base_path/model_path/injection_2_name,"w")
injection_file.writelines(lines)
injection_file.close()

# Save BG-FLOOD param file
### Parameters

In [ ]:
gpudevice = -1 
dx = 10
mask = 75
smallnc = 0
outputtimestep = 600
endtime = 10800

### Write out parameter file

In [ ]:
lines = ["## Herbet river. Parameter file. Basic test.\n", 
         "topo = " + str(dem_name) + "?z\n", 
         "gpudevice = " + str(gpudevice) + "\n", 
         "dx = " + str(dx) + "\n", 
         "mask = " + str(mask) + "\n",
         "smallnc = " + str(smallnc) + "\n",
         "outputtimestep = " + str(outputtimestep) + "\n",
         "endtime = " + str(endtime) + "\n",
         "river = " + str(injection_1_name) + ", " + str(injection_1_x1) + ", " + str(injection_1_x2) + ", " + str(injection_1_y1)
         + ", " + str(injection_1_y2) + "\n",
         "river = " + str(injection_2_name) + ", " + str(injection_2_x1) + ", " + str(injection_2_x2) + ", " +  str(injection_2_y1) 
         + ", " + str(injection_2_y2)  + "\n"] 
injection_file = open(base_path/model_path/param_name,"w")
injection_file.writelines(lines)
injection_file.close()